In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(0, "../src/")
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from handcrafted_features import DocBasedFeatureExtractor, Doc2VecChunkVectorizer, CorpusBasedFeatureExtractor
from utils import get_doc_paths, read_labels

raw_docs_dir = "../data/raw_docs/"
labels_dir = "../data/labels/"

lang = "eng"
doc_paths = get_doc_paths(raw_docs_dir, lang)

sentences_per_chunk = 200
#d2vcv = Doc2VecChunkVectorizer(lang, sentences_per_chunk)
#d2vcv.fit_transform(doc_paths)

/home/annina/anaconda3/envs/nlp/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
doc_paths = os.listdir('/home/annina/scripts/great_unread_nlp/data/processed_sentences/eng') #####
doc_paths = ['../data/raw_docs/eng/' + path for path in doc_paths][0:3]
print(doc_paths)

['../data/raw_docs/eng/Radcliffe_Ann_The-Italian_1797.txt', '../data/raw_docs/eng/Gissing_George_In-the-Year-of-Jubilee_1894.txt', '../data/raw_docs/eng/Collins_Wilkie_Armadale_1864.txt']


In [3]:
all_chunk_based_features = []
all_book_based_features = []
all_average_sbert_sentence_embeddings = []
all_doc2vec_chunk_embeddings = []
for doc_path in tqdm(doc_paths):
    print(doc_path)
    fe = DocBasedFeatureExtractor(lang, doc_path, sentences_per_chunk)
    chunk_based_features, book_based_features, average_sbert_sentence_embeddings, doc2vec_chunk_embeddings = fe.get_all_features()
    all_chunk_based_features.extend(chunk_based_features)
    all_book_based_features.append(book_based_features)
    all_average_sbert_sentence_embeddings.append(average_sbert_sentence_embeddings)
    all_doc2vec_chunk_embeddings.append(doc2vec_chunk_embeddings)

  0%|          | 0/3 [00:00<?, ?it/s]

../data/raw_docs/eng/Radcliffe_Ann_The-Italian_1797.txt
33
6672


 33%|███▎      | 1/3 [00:04<00:08,  4.03s/it]

../data/raw_docs/eng/Gissing_George_In-the-Year-of-Jubilee_1894.txt
89
17917


 67%|██████▋   | 2/3 [00:07<00:03,  3.97s/it]

../data/raw_docs/eng/Collins_Wilkie_Armadale_1864.txt
84
16885


100%|██████████| 3/3 [00:14<00:00,  4.85s/it]


In [10]:
cbfe = CorpusBasedFeatureExtractor(lang, doc_paths, all_average_sbert_sentence_embeddings, all_doc2vec_chunk_embeddings)
all_corpus_based_features = cbfe.get_all_features()

#all_corpus_based_features.to_csv('../test',index=True)

100%|██████████| 3/3 [00:01<00:00,  2.49it/s]


                                                 about        the        year  \
Radcliffe_Ann_The-Italian_1797              -77.930811  20.213771  -81.442550   
Gissing_George_In-the-Year-of-Jubilee_1894  180.758241 -45.431391  457.325001   
Collins_Wilkie_Armadale_1864                 -7.633664  21.286357  -52.421028   

                                                 some     english  \
Radcliffe_Ann_The-Italian_1797              69.071842  -95.014960   
Gissing_George_In-the-Year-of-Jubilee_1894   0.720815  -15.958928   
Collins_Wilkie_Armadale_1864               -38.324452  277.318730   

                                             travellers         in  \
Radcliffe_Ann_The-Italian_1797              8289.823058 -36.606975   
Gissing_George_In-the-Year-of-Jubilee_1894   -89.303864 -17.519647   
Collins_Wilkie_Armadale_1864                -100.000000  59.008210   

                                                 italy      during        one  \
Radcliffe_Ann_The-Italian_1797      

In [9]:
all_corpus_based_features

,the_x,to_x,of_x,and_x,i_x,a_x,in_x,he_x,that_x,you_x,...,ruddled,handsomest,cleaned,struts,superannuated,scorns,lipari,scoundrels,forborne,groundless
0,0.067713,0.032516,0.035475,0.029875,0.008844,0.017190,0.011595,0.018175,0.015227,0.008450,...,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02
1,0.036113,0.028418,0.027099,0.023026,0.022226,0.025640,0.014072,0.012768,0.012991,0.018286,...,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02
2,0.065307,0.031672,0.027480,0.026621,0.023666,0.017316,0.020166,0.014602,0.010973,0.012187,...,1.012319e+10,1.012319e+10,1.012319e+10,1.012319e+10,1.012319e+10,1.012319e+10,1.012319e+10,1.012319e+10,1.012319e+10,1.012319e+10


In [8]:
df = pd.DataFrame(all_chunk_based_features)
labels = read_labels(labels_dir, lang)
df["y"] = df["book_name"].apply(lambda x: labels[x])

../src/utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels_df["file_name"][labels_df["file_name"] == key] = value


In [14]:
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

all_predictions = []
all_labels = []

book_names = df['book_name'].unique()
book_names_splitted = np.array_split(book_names, 10)
print(book_names_splitted)
for index, split in enumerate(book_names_splitted):
    train_X = df[~df["book_name"].isin(split)].drop(columns=["y", "book_name"]).values
    print(train_X)
    train_y = df[~df["book_name"].isin(split)]["y"].values.ravel()
    validation_X = df[df["book_name"].isin(split)].drop(columns=["y", "book_name"]).values
    validation_y = df[df["book_name"].isin(split)]["y"].values.ravel()
    
    pca = PCA(n_components=150)
    train_X = pca.fit_transform(train_X)
    print(pca.explained_variance_ratio_.sum())
    validation_X = pca.transform(validation_X)
    model = XGBRegressor()
    model.fit(train_X, train_y)
    train_yhat = model.predict(train_X)
    validation_yhat = model.predict(validation_X)
    all_labels.extend(validation_y.tolist())
    all_predictions.extend(validation_yhat.tolist())
    train_mse = mean_squared_error(train_y, train_yhat)
    train_mae = mean_absolute_error(train_y, train_yhat)
    validation_mse = mean_squared_error(validation_y, validation_yhat)
    validation_mae = mean_absolute_error(validation_y, validation_yhat)
    print(f"Fold: {index+1}, TrainMSE: {train_mse}, TrainMAE: {train_mae}, ValMSE: {validation_mse}, ValMAE: {validation_mae}")
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

[array(['Radcliffe_Ann_The-Italian_1797'], dtype=object), array(['Gissing_George_In-the-Year-of-Jubilee_1894'], dtype=object), array(['Collins_Wilkie_Armadale_1864'], dtype=object), array([], dtype=object), array([], dtype=object), array([], dtype=object), array([], dtype=object), array([], dtype=object), array([], dtype=object), array([], dtype=object)]
[[ 4.25281200e-02  1.68826995e-01  3.21371184e-04 ...  5.41820377e-02
  -2.25403738e+00  1.65703654e-01]
 [ 6.00447539e-02  1.80507210e-01  2.48632521e-04 ... -5.23960032e-02
  -8.12183797e-01 -2.45996136e-02]
 [ 3.24582825e-02  1.73076923e-01  0.00000000e+00 ... -2.58206785e-01
  -7.38855898e-01  1.34449577e+00]
 ...
 [ 4.75267984e-02  1.84682939e-01  0.00000000e+00 ... -3.59380990e-01
   1.57649803e+00 -1.26107430e+00]
 [ 3.41766306e-02  1.83763535e-01  0.00000000e+00 ... -1.91105723e+00
  -9.45564568e-01  1.50816464e+00]
 [ 3.28701092e-02  1.86041741e-01  1.02558843e-04 ... -1.45941377e+00
  -3.64400268e-01  8.83179531e-02]]
0.99999

/home/annina/anaconda3/envs/nlp/lib/python3.9/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


ValueError: n_components=150 must be between 0 and min(n_samples, n_features)=117 with svd_solver='full'

In [ ]:
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.decomposition import PCA

X = df.drop(columns=['y', 'book_name']).values
y = df["y"].values.ravel()

all_predictions = []
all_labels = []
kf = KFold(n_splits=10, shuffle=True, random_state=42)
for index, (train_indices, validation_indices) in enumerate(kf.split(X)):
    train_X = X[train_indices, :]
    train_y = y[train_indices]
    validation_X = X[validation_indices, :]
    validation_y = y[validation_indices]
    pca = PCA(n_components=150)
    train_X = pca.fit_transform(train_X)
    print(pca.explained_variance_ratio_.sum())
    validation_X = pca.transform(validation_X)
    model = SVR()
    model.fit(train_X, train_y)
    train_yhat = model.predict(train_X)
    validation_yhat = model.predict(validation_X)
    all_labels.extend(validation_y.tolist())
    all_predictions.extend(validation_yhat.tolist())
    train_mse = mean_squared_error(train_y, train_yhat)
    train_mae = mean_absolute_error(train_y, train_yhat)
    validation_mse = mean_squared_error(validation_y, validation_yhat)
    validation_mae = mean_absolute_error(validation_y, validation_yhat)
    print(f"Fold: {index+1}, TrainMSE: {train_mse}, TrainMAE: {train_mae}, ValMSE: {validation_mse}, ValMAE: {validation_mae}")
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(18, 6))
plt.scatter(all_labels, all_predictions)
plt.show();

DEBUG:matplotlib:matplotlib data path: /home/annina/anaconda3/envs/nlp/lib/python3.9/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/annina/.config/matplotlib
DEBUG:matplotlib:matplotlib version 3.4.2
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', '_frozen_importlib_external', 'posix', '_io', 'marshal', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', 'encodings.latin_1', '_abc', 'abc', 'io', '__main__', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_locale', '_bootlocale', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib.machinery', '_heapq', 'heapq', 'itertools', 'keyword', '_operator', 'operator', 'reprlib', '_collections', 'collections', 'collections.abc', '_fu

DEBUG:matplotlib:CACHEDIR=/home/annina/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/annina/.cache/matplotlib/fontlist-v330.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


<Figure size 1800x600 with 1 Axes>

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


Error in callback <function _enable_matplotlib_integration.<locals>.configure_once at 0x7f7eecce7e50> (for post_run_cell):


ImportError: cannot import name 'configure_inline_support_orig' from 'matplotlib_inline.backend_inline' (/home/annina/anaconda3/envs/nlp/lib/python3.9/site-packages/matplotlib_inline/backend_inline.py)